In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import cfscrape
from lxml import etree
from datetime import date

# Initiate webdriver

In [191]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')
linkedin_search = 'https://www.linkedin.com/jobs/search/?location=Worldwide'
browser.get(linkedin_search)
actions = ActionChains(browser)

In [89]:
companies_list = pd.read_csv('Indeed_Companies-{}.csv'.format('25-08-2020'), index_col=[0])

In [ ]:
for i in companies_list.index:
    comp = companies_list['Company Original'].loc[i]
    new_comp = re.sub(r' $','', comp)
    companies_list['Company Original'].loc[i] = new_comp

In [304]:
companies_list['Company Original'].loc[91] = 'Rivian'

In [192]:
diff_comps = {'Rivian Automotive': 'Rivian',
              '100Thieves':'100 Thieves',
              'Mozilla Corp':'Mozilla',
              'RedHat':'Red Hat',
              'UnderArmour':'Under Armour',
              'ForeScout': 'Forescout Technologies Inc.',
              'William Sonoma':'Williams-Sonoma, Inc.',
              'Chef': 'Chef Software',
              'DraftKings': 'DraftKings Inc.',
              'Fanatics': 'Fanatics, Inc.',
              'StitchFix': 'Stitch Fix',
              'RealReal': 'The RealReal',
              'New Relic': 'New Relic, Inc.',
              'Synopsys': 'Synopsys Inc',
              'Twilio': 'Twilio Inc.',
              'Veeva System': 'Veeva Systems',
              'Akamai': 'Akamai Technologies',
              'Cadence Design': 'Cadence Design Systems',
              'Check Point Software': 'Check Point Software Technologies, Ltd.',
              'Coupa': 'Coupa Software',
              'Dassault Systemes': 'Dassault Systèmes',
              'Domo': 'Domo, Inc.',
              'FireEye': 'FireEye, Inc.',
              'Guidewire': 'Guidewire Software',
              'Juniper': 'Juniper Networks',
              'Okta': 'Okta, Inc.',
              'Palantir': 'Palantir Technologies',
              'Procore': 'Procore Technologies',
              'Real Page': 'RealPage, Inc.',
              'Wix': 'Wix.com',
              'Global Payments': 'Global Payments Inc.',
              'World Pay': 'Worldpay',
              'Affirm': 'Affirm, Inc.',
              'Asos': 'ASOS.com',
              'Boohoo': 'Boohoo Group PLC',
              'Zalando': 'Zalando SE',
              'Home Depot': 'The Home Depot',
              'Lowes': "Lowe's Companies, Inc.",
              'Roku': 'Roku Inc.',
              'Snap': 'Snap Inc.',
              'Monzo': 'Monzo Bank',
              'Docker': 'Docker, Inc',
              'Scale.ai': 'Scale AI',
              'Checkr': 'Checkr, Inc.',
              'Marqeta': 'Marqeta, Inc',
              'Checkout': 'Checkout.com',
              'Sourcefire':'Sourcefire, part of Cisco',
              'Agora.io':'Agora',
              'Aras':'Aras Corporation',
              'GitLab':'GitLab Inc.',
              'Pentaho':'Pentaho from Hitachi Vantara',
              'Mercury.co': 'Mercury - Banking for Startups'}
len(diff_comps)

52

In [163]:
companies_list['Company LinkedIn'] = companies_list['Company Original']
for i in companies_list.index:
    comp = companies_list['Company Original'].loc[i]
    try:
        companies_list['Company LinkedIn'].loc[i] = diff_comps[comp]
    except:
        pass

In [164]:
companies_list[companies_list['Company LinkedIn']!=companies_list['Company Original']].shape

(52, 11)

In [424]:
companies_list.to_csv('Indeed_Companies-{}.csv'.format(date.today().strftime("%d-%m-%Y")))

In [204]:
missed_comps = [x for x in companies_list['Company Original'] if x not in linkedin_posts['Company'].unique()]
len(missed_comps)

19

# Search

In [22]:
wrong_choice = []
temp_diff_comps = {}

In [15]:
#linkedin_posts = pd.DataFrame([])

In [307]:
%%time
start = 1
for i, company in enumerate(missed_comps[start:], start = start):
    
    
    print('\n#', i, '|', company)
    print('-'*80)
    
    sleep(2)
    # Click Company
    path = '/html/body/header/section/form/ul/li[3]/div/button'
    browser.find_element_by_xpath(path).click()
    sleep(1)
    
    # Send company
    search_comp = companies_list[companies_list['Company Original'] == company]['Company LinkedIn'].values[0]
    
    path = '/html/body/header/section/form/ul/li[3]/div/div/fieldset/div[1]/div/input'
    browser.find_element_by_xpath(path).send_keys(search_comp)
    sleep(1)
    browser.find_element_by_xpath(path).send_keys(Keys.BACKSPACE)

    sleep(1)
    browser.find_element_by_xpath(path).send_keys(search_comp[-1])
    
    
    # Choose company
    try:
        sleep(1)
        path = '/html/body/header/section/form/ul/li[3]/div/div/fieldset/div[1]/div/input'
        browser.find_element_by_xpath(path).send_keys(Keys.ARROW_DOWN)
        
        sleep(3)
        browser.find_element_by_xpath(path).send_keys(Keys.ENTER)
        sleep(1)
    except:
        pass
    try:
        # Press search
        path = '/html/body/header/section/form/ul/li[3]/div/div/fieldset/div[2]/button'
        browser.find_element_by_xpath(path).click()
    except:
        print('Wrong Company')
        wrong_choice.append(company)
        
        try:
            # Clear
            path = '//*[@class="top-filters__clear-button"]'
            browser.find_element_by_xpath(path).click()
            continue
        except:
            continue
    
    # Matched
    sleep(1)
    try:
        path = '//*[@class="no-results__text"]'
        no_match = browser.find_element_by_xpath(path).text
        if no_match:
            print(no_match)
            
            try:
                # Clear
                path = '//*[@class="top-filters__clear-button"]' 
                browser.find_element_by_xpath(path).click()
                continue
            
            except:
                continue
    except:
        pass
    
    sleep(2)
    try:
        # Got the right company?
        path = '/html/body/main/div/section/ul/li[{}]'.format(1)
        got_company = browser.find_element_by_xpath(path).text.split('\n')[1]
        if got_company.lower() == search_comp.lower() or got_company in diff_comps.values():
            pass
        
        else:
            print('Wrong Company')
            temp_diff_comps[company] = got_company
            
            try:
                # Clear
                path = '//*[@class="top-filters__clear-button"]' 
                browser.find_element_by_xpath(path).click()
                continue
            
            except:
                continue
    
    except:
        print('No postings in search results')
        wrong_choice.append(company)
        
        try:
            # Clear
            path = '//*[@class="top-filters__clear-button"]' 
            browser.find_element_by_xpath(path).click()
            continue
        except:
            continue
    
    
    
    # Results
    path = '//*[@class="results-context-header__job-count"]'
    results = browser.find_element_by_xpath(path).text
    results = re.sub(r'[^\d]','',results)
    results = int(results)
   
    print('+ Number of postings:', results)
    
    # Sorting
    sleep(2)
    if results >=1000:
        path = '/html/body/header/section/form/ul/li[1]/div/button'
        browser.find_element_by_xpath(path).click()
        sleep(1)
        path = '/html/body/header/section/form/ul/li[1]/div/div/fieldset/div[1]/ul/li[2]/label'
        browser.find_element_by_xpath(path).click()
        sleep(1)
        path = '/html/body/header/section/form/ul/li[1]/div/div/fieldset/div[2]/button'
        browser.find_element_by_xpath(path).click()
        sleep(1)
    else:
        pass
    
    sleep(2)
    # Scrolling pages
    for i in range(50):

        for _ in range(1):
            actions.send_keys(Keys.PAGE_DOWN).perform()
        
        sleep(0.3)
        # More    
        try:
            path = '//*[@class="infinite-scroller__show-more-button infinite-scroller__show-more-button--visible"]'
            browser.find_element_by_xpath(path).click()
            sleep(0.3)
        except:
            pass

        sleep(0.3)
            

        # Reached the bottom
        try:
            path = '//*[@class="inline-notification__text"]'
            bottom = browser.find_element_by_xpath(path).text
            if bottom:
                sleep(1)
                print('+', bottom)
                break
        except:
            pass
    
    # Populating Company dataframe
    sleep(2)
    print('+', 'Getting Company postings...')
    if results > 1000:
        results=1000
    
    # Company Data
    company_postings = pd.DataFrame(range(results), columns=['Company'])
    company_postings['Company'] = company
    company_postings['Posting URL'] = np.nan
    company_postings['Meta'] = np.nan
    company_postings['Number of Postings'] = results
    company_postings['Date'] = date.today().strftime("%d/%m/%Y")
    
    for row in range(1, results+1):
        sleep(0.05)
        if row%200==0:
            print('got', row, 'out of', results,)
        try:  
            path = '/html/body/main/div/section/ul/li[{}]'.format(row)
            meta = browser.find_element_by_xpath(path).text
            company_postings['Meta'].loc[row-1] = meta


            path = '/html/body/main/div/section/ul/li[{}]/a'.format(row)
            post_url = browser.find_element_by_xpath(path).get_attribute('href')
            company_postings['Posting URL'].loc[row-1] = post_url
        except:
            pass
    
    sleep(1)
    print('+', 'Missed data, %')
    missed_data = round(company_postings[['Meta','Posting URL']].isna().sum()/company_postings.shape[0]*100, 1)
    print(missed_data)
    
    linkedin_posts = pd.concat([linkedin_posts, company_postings], ignore_index=True)
    print('\n+', 'Whole Table Size', linkedin_posts.shape[0])
    
    sleep(1)
    # Clear
    path = '/html/body/header/section/form/ul/li[6]/button'
    browser.find_element_by_xpath(path).click()
    sleep(1)


# 1 | Rivian
--------------------------------------------------------------------------------
+ Number of postings: 579
+ Getting Company postings...
got 200 out of 579
got 400 out of 579
+ Missed data, %
Meta           0.7
Posting URL    0.7
dtype: float64

+ Whole Table Size 49018

# 2 | Shift
--------------------------------------------------------------------------------
+ Number of postings: 59
+ You've viewed all jobs for this search
+ Getting Company postings...
+ Missed data, %
Meta           0.0
Posting URL    0.0
dtype: float64

+ Whole Table Size 49077
CPU times: user 3.19 s, sys: 227 ms, total: 3.42 s
Wall time: 2min 52s


In [207]:
linkedin_posts.shape

(48394, 5)

In [308]:
linkedin_posts['Company'].nunique()

210

In [209]:
linkedin_posts.to_csv('Indeed_LinkedIn-Date-{}.csv'.format(date.today().strftime("%d-%m-%Y")))

## Processing data

In [309]:
linkedin_processed = linkedin_posts.copy()

In [311]:
linkedin_processed['Job Title'] = linkedin_processed['Meta'].str.split('\n', expand=True)[0]
linkedin_processed['Post Company'] = linkedin_processed['Meta'].str.split('\n', expand=True)[1]
linkedin_processed['Job Place'] = linkedin_processed['Meta'].str.split('\n', expand=True)[2]
linkedin_processed['Posting Date'] = linkedin_processed['Meta'].str.extract(r'(.+(?= ago))', expand=True)[0]

In [312]:
linkedin_processed.isna().sum()

Company                  0
Posting URL           5927
Meta                  5926
Number of Postings       0
Date                     0
Job Title             5926
Post Company          5926
Job Place             5926
Posting Date          5952
dtype: int64

In [313]:
linkedin_processed[(linkedin_processed.duplicated(subset=['Posting URL'])) &
                   (~linkedin_processed['Meta'].isna())].shape

(1, 9)

In [333]:
num_posts = linkedin_processed[~linkedin_processed['Meta'].isna()].groupby(['Company'])['Meta'].count()
num_posts = num_posts.reset_index().rename(columns={'Meta':'Postings Parsed'})

In [334]:
linkedin_processed = linkedin_processed.drop(columns='Postings Parsed')

In [335]:
linkedin_processed = pd.merge(linkedin_processed, num_posts, on='Company', how='left')

In [317]:
linkedin_processed.head(1)

,Company,Posting URL,Meta,Number of Postings,Date,Job Title,Post Company,Job Place,Posting Date,Postings Parsed
0,8X8,https://uk.linkedin.com/jobs/view/solutions-co...,Solutions Consultant - remote\n8x8\nManchester...,71,25/08/2020,Solutions Consultant - remote,8x8,"Manchester, England, United Kingdom",20 hours,96


In [336]:
linkedin_processed['Number of Postings'] = linkedin_processed['Number of Postings'].astype(float)

In [337]:
linkedin_processed['Postings Delta'] = linkedin_processed['Postings Parsed'] - linkedin_processed['Number of Postings']
linkedin_processed['Postings Delta'] = round(linkedin_processed['Postings Delta']/linkedin_processed['Number of Postings']*100, 1)

In [339]:
linkedin_processed[(linkedin_processed['Postings Delta'] <= -3) &
                   (~linkedin_processed.duplicated(subset=['Company'], keep='first'))].shape#sample(2)

(3, 15)

In [240]:
linkedin_processed[linkedin_processed['Meta'].isna()]['Company'].nunique()#.sample(10)

13

#### Processing dates

In [322]:
linkedin_processed['Posting Date Multiplier'] = linkedin_processed['Posting Date']
for i in linkedin_processed.index:
    if 'week' in str(linkedin_processed['Posting Date Multiplier'].loc[i]):
        linkedin_processed['Posting Date Multiplier'].loc[i] = 7
    
    if 'month' in str(linkedin_processed['Posting Date Multiplier'].loc[i]):
        linkedin_processed['Posting Date Multiplier'].loc[i] = 30
    
    if 'year' in str(linkedin_processed['Posting Date Multiplier'].loc[i]):
        linkedin_processed['Posting Date Multiplier'].loc[i] = 365
    
    if 'day' in str(linkedin_processed['Posting Date Multiplier'].loc[i]):
        linkedin_processed['Posting Date Multiplier'].loc[i] = 1
    
    if 'hour' in str(linkedin_processed['Posting Date Multiplier'].loc[i]):
        linkedin_processed['Posting Date Multiplier'].loc[i] = 1
    
    if 'minute' in str(linkedin_processed['Posting Date Multiplier'].loc[i]):
        linkedin_processed['Posting Date Multiplier'].loc[i] = 1

In [323]:
linkedin_processed['Posting Date Multiplier'].value_counts()

1      20205
7      17826
30      5030
365       64
Name: Posting Date Multiplier, dtype: int64

In [324]:
linkedin_processed['Posting Date Multiplier'] = linkedin_processed['Posting Date Multiplier'].astype(float)

In [325]:
linkedin_processed['Posting Date Num Raw'] = linkedin_processed['Posting Date']
for i in linkedin_processed.index:

    if 'hour' in str(linkedin_processed['Posting Date Num Raw'].loc[i]):
        linkedin_processed['Posting Date Num Raw'].loc[i] = 1
    
    if 'minute' in str(linkedin_processed['Posting Date Num Raw'].loc[i]):
        linkedin_processed['Posting Date Num Raw'].loc[i] = 1


In [326]:
linkedin_processed['Posting Date Pre Num'] = linkedin_processed['Posting Date Num Raw'].astype(str).str.extract(r'(\d)').astype(float)

In [327]:
linkedin_processed['Posting Date Num'] = linkedin_processed['Posting Date Pre Num']*linkedin_processed['Posting Date Multiplier']

In [423]:
linkedin_processed[['Posting Date', 'Posting Date Multiplier', 
                   'Posting Date Pre Num', 'Posting Date Num']].sample(3)

,Posting Date,Posting Date Multiplier,Posting Date Pre Num,Posting Date Num
7712,6 days,1.0,6.0,6.0
20953,4 weeks,7.0,4.0,28.0
9034,1 week,7.0,1.0,7.0


In [329]:
linkedin_processed.isna().sum()/linkedin_processed.shape[0]*100

Company                     0.000000
Posting URL                12.076940
Meta                       12.074903
Number of Postings          0.000000
Date                        0.000000
Job Title                  12.074903
Post Company               12.074903
Job Place                  12.074903
Posting Date               12.127881
Postings Parsed             0.000000
Postings Delta              0.000000
Posting Date Multiplier    12.127881
Posting Date Num Raw       12.127881
Posting Date Pre Num       12.127881
Posting Date Num           12.127881
dtype: float64

In [330]:
linkedin_processed.shape

(49077, 15)

In [332]:
linkedin_processed = linkedin_processed.drop_duplicates(subset=['Posting URL']).dropna(subset=['Posting URL'])
linkedin_processed.shape

(43150, 15)

## Getting correct titles

### Classifying Titles

In [340]:
linkedin_processed['Job Title'].nunique()

20795

In [341]:
linkedin_processed['Job Title New'] = linkedin_processed['Job Title'].fillna('-999').astype(str)

In [342]:
# sales
sales_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'sale'))]['Job Title New'].index.to_list()
linkedin_processed['Job Title New'].loc[sales_idx] = 'sales'
linkedin_processed['Job Title New'].nunique()

18616

In [343]:
# engineering
eng_idx = linkedin_processed[linkedin_processed['Job Title New'].str.lower().str.contains(r' engineer(?![a-z])|engineer |develop|architect')].index.to_list()
linkedin_processed['Job Title New'].loc[eng_idx] = 'engineering'
linkedin_processed['Job Title New'].nunique()

12707

In [344]:
# program & product management
prod_idx = linkedin_processed[((linkedin_processed['Job Title New'].str.lower().str.contains(r' manager(?![a-z])') |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'manager |team lead')))) &

                 ((linkedin_processed['Job Title New'].str.lower().str.contains(r' product(?![a-z])') |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'product ')))) |

                 ((linkedin_processed['Job Title New'].str.lower().str.contains(r' program(?![a-z])') |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'program ')))) | 
                
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'product owner|product specialist'))

                ]['Job Title New'].index.to_list()
linkedin_processed['Job Title New'].loc[prod_idx] = 'program & product management'
linkedin_processed['Job Title New'].nunique()

11696

In [345]:
# analytics
analytics_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'scientist')) | 
                     (linkedin_processed['Job Title New'].str.lower().str.contains(r'research')) |
                     (linkedin_processed['Job Title New'].str.lower().str.contains(r'analys'))
                    ]['Job Title New'].index.to_list()

linkedin_processed['Job Title New'].loc[analytics_idx] = 'analytics'
linkedin_processed['Job Title New'].nunique()

10486

In [346]:
# design
design_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'design'))]['Job Title New'].index.to_list()
linkedin_processed['Job Title New'].loc[design_idx] = 'design'
linkedin_processed['Job Title New'].nunique()

10131

In [347]:
# consulting
consult_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'consultant|advis'))]['Job Title New'].index.to_list()
linkedin_processed['Job Title New'].loc[consult_idx] = 'consulting'
linkedin_processed['Job Title New'].nunique()
#.value_counts()#.nunique()##.nunique()


9484

In [348]:
# real estate
re_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'real estate|closer'))]['Job Title New'].index.to_list()
linkedin_processed['Job Title New'].loc[re_idx] = 'real estate'
linkedin_processed['Job Title New'].nunique()

9243

In [349]:
# operational
oper_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'support')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'inventory')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'representative|executive assistant')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'office associate')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'fulfillment')) |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'administr')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'specialist')) |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'coordinator|buyers assistant')) |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'operations|supply associate')) 
                

                ]['Job Title New'].index.to_list()

linkedin_processed['Job Title New'].loc[oper_idx] = 'operational'
linkedin_processed['Job Title New'].nunique()

7418

In [363]:
# techicians & general labor
general_idx = linkedin_processed[
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'operator|technical|mechanic')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'lot attendant|freight\/receiving|auto body shop')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'warehouse worker|deliver|livreur')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'roofer|vehicle condition assessor')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'installer|detailer|electrician|worker')) |

                (linkedin_processed['Job Title New'].str.lower().str.contains(r'painter|concierge|driver|cashier')) |

    
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'retail store associate|courier')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'warehouse store associate|stock')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'warehouse associate|lot associate')) |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'supervisor|librarian|retail associate')) |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'technici|general labor')) 
                ]['Job Title New'].index.to_list()

linkedin_processed['Job Title New'].loc[general_idx] = 'technicians & general labor'
linkedin_processed['Job Title New'].nunique()

1430

In [351]:
# general management
mng_idx = linkedin_processed[((linkedin_processed['Job Title New'].str.lower().str.contains(r' manager(?![a-z])') |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'manager |manager\,|lead')))) &
               
               

                 (~linkedin_processed['Job Title New'].fillna('-999').astype(str).str.lower().str.contains(r' product(?![a-z])') &
                 (~linkedin_processed['Job Title New'].fillna('-999').astype(str).str.lower().str.contains(r'product '))) &


                 (~linkedin_processed['Job Title New'].fillna('-999').astype(str).str.lower().str.contains(r' program(?![a-z])') &
                 (~linkedin_processed['Job Title New'].fillna('-999').astype(str).str.lower().str.contains(r'program ')))]['Job Title New'].index.to_list()

linkedin_processed['Job Title New'].loc[mng_idx] = 'general management'
linkedin_processed['Job Title New'].nunique()

3319

In [352]:
# executives
exec_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'director')) | 
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'head of')) |
                (linkedin_processed['Job Title New'].str.lower().str.contains(r'partner')) | 
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'president')) |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'officer')) |
                (linkedin_processed['Job Title New'].str.contains(r'VP')) |
                 (linkedin_processed['Job Title New'].str.lower().str.contains(r'executiv')) 
                
                
                ]['Job Title New'].index.to_list()

linkedin_processed['Job Title New'].loc[exec_idx] = 'top/middle-level management'
linkedin_processed['Job Title New'].nunique()

1900

In [353]:
# HR
hr_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'hr|human resou|hiring|recruit|talent|trainer'))]['Job Title New'].index.to_list()
linkedin_processed['Job Title New'].loc[hr_idx] = 'human resources'
linkedin_processed['Job Title New'].nunique()

1803

In [354]:
#linkedin_processed['Job Title New'].str.split(pat = " ", expand=True).stack().value_counts()

In [355]:
#linkedin_processed[linkedin_processed['Job Title New'].str.lower().str.contains('stock')]['Job Title New'].value_counts()

In [356]:
# marketing
marketing_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'marketing|copywriter|marketer|content|merchandising'))]['Job Title New'].index.to_list()
linkedin_processed['Job Title New'].loc[marketing_idx] = 'marketing'
linkedin_processed['Job Title New'].nunique()

1711

In [357]:
# Finance
finance_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'accountant|financ|payroll|mortgage|risk|tax|impuestos'))]['Job Title New'].index.to_list()
linkedin_processed['Job Title New'].loc[finance_idx] = 'finance'
linkedin_processed['Job Title New'].nunique()

1584

In [358]:
# Legal
legal_idx = linkedin_processed[(linkedin_processed['Job Title New'].str.lower().str.contains(r'legal|counsel'))]['Job Title New'].index.to_list()
linkedin_processed['Job Title New'].loc[legal_idx] = 'legal'
linkedin_processed['Job Title New'].nunique()

1440

#### Coverage

In [359]:
linkedin_processed = linkedin_processed[linkedin_processed['Job Title New'] !='-999']

In [422]:
linkedin_processed['Job Title New'].value_counts().head(17)

engineering                     13658
general management               4666
technicians & general labor      4644
sales                            4007
operational                      3256
top/middle-level management      2507
analytics                        2180
program & product management     2093
other                            2086
consulting                       1459
finance                          1047
design                            684
real estate                       258
legal                             229
marketing                         208
human resources                   168
Name: Job Title New, dtype: int64

In [365]:
main_titles = linkedin_processed['Job Title New'].value_counts().head(15).index.to_list()
linkedin_processed[linkedin_processed['Job Title New'].isin(main_titles)].shape[0]/linkedin_processed.shape[0]

0.951657010428737

In [366]:
# Other Titles
other_idx = linkedin_processed[~linkedin_processed['Job Title New'].isin(main_titles)].index.to_list()
linkedin_processed['Job Title New'].loc[other_idx] = 'other'
linkedin_processed['Job Title New'].nunique()

16

In [367]:
linkedin_processed['Job Title New'].value_counts()

engineering                     13658
general management               4666
technicians & general labor      4644
sales                            4007
operational                      3256
top/middle-level management      2507
analytics                        2180
program & product management     2093
other                            2086
consulting                       1459
finance                          1047
design                            684
real estate                       258
legal                             229
marketing                         208
human resources                   168
Name: Job Title New, dtype: int64

### Missed Companies

In [369]:
missed_compnaies = []
for company in companies_list['Company Original'].unique():
    if company not in linkedin_processed['Company'].unique():# and company not in no_jobs: # 
        missed_compnaies.append(company)
len(missed_compnaies)

18

In [371]:
no_jobs = missed_compnaies
len(no_jobs)

18

In [372]:
no_jobs_df = pd.DataFrame(no_jobs, columns=['Company'])

## Getting weekly stat

In [373]:
linkedin_processed = pd.concat([linkedin_processed, no_jobs_df], axis=0, ignore_index=True)

In [426]:
linkedin_processed.to_csv('Indeed_LinkedIn-Processed-{}.csv'.format(date.today().strftime("%d-%m-%Y")))

### First two weeks

In [374]:
linkedin_processed['Company'].nunique()

228

In [375]:
groupe_df = linkedin_processed[linkedin_processed['Posting Date Num']<=15].groupby(['Company','Job Title New'])['Job Title'].count()
groupe_df = groupe_df.reset_index()
groupe_df.shape

(1795, 3)

In [376]:
pivot_df_two =pd.pivot_table(groupe_df, values='Job Title', index=['Company'],
                        columns=['Job Title New'], aggfunc=np.sum).reset_index()
pivot_df_two.shape

(207, 17)

In [377]:
pivot_df_two['Total'] = pivot_df_two.loc[:,'analytics':'top/middle-level management'].sum(axis=1)

In [378]:
pivot_df_two['Company'].nunique()

207

In [379]:
# missed comps in pivot
missed_in_pivot = [x for x in linkedin_processed['Company'].unique() if x not in pivot_df_two['Company'].unique()]
missed_in_pivot_df_two = pd.DataFrame(missed_in_pivot, columns=['Company'])


In [380]:
pivot_df_two = pd.concat([pivot_df_two, missed_in_pivot_df_two], ignore_index=True)
pivot_df_two.shape

(228, 18)

In [382]:
pivot_df_two = pivot_df_two.sort_values('Company')
pivot_df_two.shape

(228, 18)

In [383]:
cols = ['Company', 'Total','analytics', 'consulting', 'design', 'engineering',
       'finance', 'general management', 'human resources', 'legal',
       'marketing', 'operational', 'other', 'program & product management',
       'real estate', 'sales', 'technicians & general labor',
       'top/middle-level management']
pivot_df_two = pivot_df_two[cols]

In [387]:
spreadsheet_key = '14Q8hgWrH-OpEEIky8yB2zuIKGEJXNkfjspqogFwGXuw'

In [388]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)

In [389]:
d2g.upload(pivot_df_two.fillna('-').reset_index(drop=True),
           spreadsheet_key,
           '0 to 15 days back. {}'.format(date.today().strftime("%d-%m-%Y")),
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet '0 to 15 days back. 26-08-2020' id:1956497962>

### Second two weeks

In [390]:
groupe_df = linkedin_processed[(linkedin_processed['Posting Date Num']>=15) & (linkedin_processed['Posting Date Num']<=30)].groupby(['Company','Job Title New'])['Job Title'].count()
groupe_df = groupe_df.reset_index()
groupe_df.shape

(1282, 3)

In [391]:
pivot_df_two =pd.pivot_table(groupe_df, values='Job Title', index=['Company'],
                        columns=['Job Title New'], aggfunc=np.sum).reset_index()
pivot_df_two.shape

(199, 17)

In [392]:
pivot_df_two['Total'] = pivot_df_two.loc[:,'analytics':'top/middle-level management'].sum(axis=1)

In [393]:
pivot_df_two['Company'].nunique()

199

In [394]:
# missed comps in pivot
missed_in_pivot = [x for x in linkedin_processed['Company'].unique() if x not in pivot_df_two['Company'].unique()]
missed_in_pivot_df_two = pd.DataFrame(missed_in_pivot, columns=['Company'])


In [395]:
pivot_df_two = pd.concat([pivot_df_two, missed_in_pivot_df_two], ignore_index=True)
pivot_df_two.shape

(228, 18)

In [396]:
pivot_df_two = pivot_df_two.sort_values('Company')
pivot_df_two.shape

(228, 18)

In [397]:
cols = ['Company', 'Total','analytics', 'consulting', 'design', 'engineering',
       'finance', 'general management', 'human resources', 'legal',
       'marketing', 'operational', 'other', 'program & product management',
       'real estate', 'sales', 'technicians & general labor',
       'top/middle-level management']
pivot_df_two = pivot_df_two[cols]

In [398]:
d2g.upload(pivot_df_two.fillna('-').reset_index(drop=True),
           spreadsheet_key,
           '15 to 30 days back. {}'.format(date.today().strftime("%d-%m-%Y")),
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet '15 to 30 days back. 26-08-2020' id:272044683>

### More than month

In [412]:
groupe_df = linkedin_processed[linkedin_processed['Posting Date Num']>=30].groupby(['Company','Job Title New'])['Job Title'].count()
groupe_df = groupe_df.reset_index()
groupe_df.shape

(636, 3)

In [413]:
pivot_df_two =pd.pivot_table(groupe_df, values='Job Title', index=['Company'],
                        columns=['Job Title New'], aggfunc=np.sum).reset_index()
pivot_df_two.shape

(135, 17)

In [414]:
pivot_df_two['Total'] = pivot_df_two.loc[:,'analytics':'top/middle-level management'].sum(axis=1)

In [415]:
pivot_df_two['Company'].nunique()

135

In [416]:
# missed comps in pivot
missed_in_pivot = [x for x in linkedin_processed['Company'].unique() if x not in pivot_df_two['Company'].unique()]
missed_in_pivot_df_two = pd.DataFrame(missed_in_pivot, columns=['Company'])


In [417]:
pivot_df_two = pd.concat([pivot_df_two, missed_in_pivot_df_two], ignore_index=True)
pivot_df_two.shape

(228, 18)

In [418]:
pivot_df_two = pivot_df_two.sort_values('Company')
pivot_df_two.shape

(228, 18)

In [419]:
cols = ['Company', 'Total','analytics', 'consulting', 'design', 'engineering',
       'finance', 'general management', 'human resources', 'legal',
       'marketing', 'operational', 'other', 'program & product management',
       'real estate', 'sales', 'technicians & general labor',
       'top/middle-level management']
pivot_df_two = pivot_df_two[cols]

In [420]:
d2g.upload(pivot_df_two.fillna('-').reset_index(drop=True),
           spreadsheet_key,
           '30+ days back. {}'.format(date.today().strftime("%d-%m-%Y")),
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet '30+ days back. 26-08-2020' id:1231174472>

### Raw Data

In [411]:
raw_data = linkedin_processed[['Company',
                              'Job Title',
                              'Posting Date',
                              'Job Title New',
                              'Job Place',
                              'Posting URL']].fillna('-')
raw_data.shape

(43168, 6)

In [409]:
d2g.upload(raw_data,
           spreadsheet_key,
           'Raw Data. {}'.format(date.today().strftime("%d-%m-%Y")),
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet 'Raw Data. 26-08-2020' id:986302769>